## Tell me a song, 
## I will tell you if it is hot, 
## if not I will recomend you a similar song.

![Pokemon](images/spotify.png)

In [140]:
user_choice = input(str('Tell me a song : ')).title()
check(user_choice )

Tell me a song : halo
Right now it is not a hot song, so tell my the link of a playlist,
I'll recomend you a song based on this playlist,
it will take me a few moments : https://open.spotify.com/playlist/5XbnEtKOWl1c2GwfY1bnN8?si=03577c741a7f4de0
Probably you will like  Main Agar from  Pritam enjoy it.


In [ ]:
#Link of an exaple play list (with 13 songs it takes like 15 mins to run it).
'https://open.spotify.com/playlist/5XbnEtKOWl1c2GwfY1bnN8?si=03577c741a7f4de0'

In [130]:
# 1.Fuction.
def check(selected_song):
        if selected_song in songs['title'].values:  
            recomendation = songs.sample()
            print('It is in the list, probaly you will like',recomendation['title'].values[0], 'from',recomendation['artist'].values[0])
        else: 
            link = input("Right now it is not a hot song, so tell my the link of a playlist,\n" 
                         "I'll recomend you a song based on this playlist,\n"
                         "it will take me a few moments : ")
            code = re.findall('\w{20,25}', str(link))
            playlist = sp.user_playlist_tracks("spotify", code[0])
            playlist_more_song_features(playlist)
            uri = sp.search(q=selected_song)['tracks']['items'][0]['uri']
            user_song_feat = sp.audio_features(uri)
            user_song_feat = pd.DataFrame(user_song_feat).drop(['id','uri','track_href','analysis_url','type'],axis=1)
            user_song_feat_sca = scale.transform(user_song_feat)
            user_clus = kmeans.predict(user_song_feat_sca)  
            
            recomendation = (song_clusters[song_clusters['cluster'] == user_clus[0]]).sample()
            
            print('Probably you will like ',
                  (recomendation['song'].values)[0],
                 'from ', (recomendation['artists'].values)[0],"enjoy it.")

In [127]:
# 1.Fuction.
def check(selected_song):
        if selected_song in songs['title'].values:  
            recomendation = songs.sample()
            print('It is in the list, probaly you will like',recomendation['title'].values[0], 'from',recomendation['artist'].values[0])
        else: 
            link = input("Right now it is not a hot song, so tell my the link of a playlist,\n" 
                         "I'll recomend you a song based on this playlist,\n"
                         "it will take me a few moments : ")
            code = re.findall('\w{20,25}', str(link))
            playlist = sp.user_playlist_tracks("spotify", code[0])
            playlist_more_song_features(playlist)
            uri = sp.search(q=selected_song)['tracks']['items'][0]['uri']
            user_song_feat = sp.audio_features(uri)
            user_song_feat = pd.DataFrame(user_song_feat).drop(['id','uri','track_href','analysis_url','type'],axis=1)
            user_song_feat_sca = scale.transform(user_song_feat)
            user_clus = kmeans.predict(user_song_feat_sca)  
            print('Probably you will like ',
                  (song_clusters[song_clusters['cluster'] == user_clus[0]]['song'].sample().values)[0],
                 'from ', (song_clusters[song_clusters['cluster'] == user_clus[0]]['artists'].sample().values)[0]," enjoy it")

### 1.Getting all libraries.

In [134]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import random
import re
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

### 2. Web scraping to have a data set of top songs.

In [135]:
url = 'https://www.billboard.com/charts/hot-100'
response = requests.get(url)
response.status_code
soup = BeautifulSoup(response.content, "html.parser")

# To generate the data frame of all top songs.
num_itr = len(soup.select('span.chart-element__information > span.chart-element__information__song.text--truncate.color--primary'))

rank = []
title = []
art = []

for song in range(num_itr):
    title.append(soup.select('span.chart-element__information__song')[song].get_text())
    art.append(soup.select('span.chart-element__information__artist')[song].get_text())
    rank.append(soup.select('span.chart-element__rank__number')[song].get_text())
songs = pd.DataFrame({"ranking":rank, "title":title, "artist":art})  

### 3. Create a collection of songs with their audio features, depending on a link to give the recomendation.

In [136]:
secrets_file = open("../../spotify_user.txt","r")
string = secrets_file.read()
string.split('\n')
secrets_dict={}
for line in string.split('\n'):
  secrets_dict[line.split(':')[0]]=line.split(':')[1]

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['client_id'],
                                                           client_secret=secrets_dict['client_secret']))

In [137]:
def playlist_more_song_features(playlist):    
    songs = get_name_artists_from_playlist(playlist) 

    # In order to get all the singers in a list.
    singers = []
    for i in range(len(songs)):             
        singers.append(songs[i][1])

    singers = list(dict.fromkeys(singers))  #To drop duplicates singers

    # Itereting with the previous fuction, we have names of more songs in a list
    more_songs = []   
    for i in range(len(singers)):  
        more_songs.append(get_artist_tracks(singers[i]))

    more_songs = flatten(more_songs)  # To have it in a singel list

    songs_names = []
    artists_names = []
    uris = []

    # To have the songs and artists names and features
    for track in more_songs:
        songs_names.append(track['name'])
        artists_names.append(track['artists'][0]['name'])

    i = 0
    chunk_of_100_uris = []
    for track in more_songs:
        if i < 100:
            chunk_of_100_uris.append(sp.audio_features(track['uri']))
            i = i+1
        else:
            uris.append(chunk_of_100_uris)
            i = 0
            chunk_of_100_uris = []
            chunk_of_100_uris.append(sp.audio_features(track['uri']))
            i = i+1       
    uris.append(chunk_of_100_uris)   

    uris = flatten(flatten(uris))  # To have a unique list

    #Creating the data Frame
    S = pd.DataFrame(songs_names)
    A = pd.DataFrame(artists_names)
    U = pd.DataFrame(uris)

    S.columns = ['song']
    A.columns = ['artists']

    SAU = pd.concat([S,A,U],axis=1)
    SAU = SAU.drop(['id','uri','track_href','analysis_url','type'],axis=1)


    #Spliting the data frame
    SAU_name = SAU[['song','artists']]
    SAU_features = SAU.drop(['song','artists'],axis=1)

    # Normalizing it.
    scale = StandardScaler()
    X_prep = scale.fit_transform(SAU_features)

    # The prediction model.
    kmeans = KMeans(n_clusters=12, random_state=1234)
    kmeans.fit(X_prep)

    # Predicting / assigning the clusters:
    clusters = kmeans.predict(X_prep)

    #Finall data frame
    clusters = pd.DataFrame(clusters, columns = ['cluster'])
    SAU_features_stand = pd.DataFrame(X_prep,columns = SAU_features.columns)
    song_clusters = pd.concat([clusters,SAU_name,SAU_features_stand],axis=1)
    song_clusters = song_clusters.drop_duplicates(['song','artists'])

### 4. Fuctions

In [138]:
def get_name_artists_from_track(track):
    return [(track["name"],track['artists'][0]['name'])]

def get_name_artist_from_playlist_item(playlist_item):
      return get_name_artists_from_track(playlist_item['track'])
    
def flatten(input_list):
      return [item for sublist in input_list for item in sublist]
    
def get_name_artists_from_playlist(input_playlist):
      return flatten(list(map(get_name_artist_from_playlist_item,input_playlist["items"])))

In [139]:
# Defining a fuction to be able to get 950 songs of each singer in the previous list.
def get_artist_tracks(artist):  
    results = sp.search(q=artist,limit=50)
    tracks = results['tracks']['items']
    i=0
    while ((results['tracks']['next']!=None)) & (i<19):
        i = i+1
        results = sp.next(results['tracks'])
        tracks = tracks + results['tracks']['items']
    return tracks